<a href="https://colab.research.google.com/github/OmdenaAI/OmdenaKnowledge_AIAgentsInferenceBenchmarking/blob/faraaz/crewai_agent_benchmark.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install crewai==0.28.8 crewai_tools==0.1.6 langchain_community==0.0.29

## Loading trained model

In [ ]:
from crewai import Agent, Task, Crew
from langchain_community.chat_models import ChatCohere
from openai import OpenAI
from transformers import pipeline
import joblib
import re
import string
import pandas as pd

# Load trained ML model (Logistic Regression as an example)
ml_model = joblib.load("disaster_model.pkl")
vectorizer = joblib.load("vectorizer.pkl")

/usr/local/lib/python3.11/dist-packages/pydantic/_internal/_generate_schema.py:775: UserWarning: Mixing V1 models and V2 models (or constructs, like `TypeAdapter`) is not supported. Please upgrade `CrewAgentExecutor` to V2.
  warn(


## Use CrewAI Agents

In [ ]:
from crewai import Agent, Task, Crew
from langchain_community.chat_models import ChatCohere
import os
import joblib
import re
import string


os.environ["COHERE_API_KEY"] = userdata.get('COHERE_API_KEY')
llm = ChatCohere()


# Load trained ML model & vectorizer
ml_model = joblib.load("disaster_model.pkl")
vectorizer = joblib.load("vectorizer.pkl")

# Define AI Agents
text_processor = Agent(
    role="Text Processor",
    goal="Clean and prepare tweet text for disaster analysis.",
    backstory="You're responsible for preprocessing tweet text. "
              "You ensure text is properly formatted, lowercased, and free from unnecessary symbols.",
    allow_delegation=False,
    verbose=True,
    llm=llm,  # Now using LLM to process tasks
)

disaster_classifier = Agent(
    role="Disaster Classifier",
    goal="Classify tweets as disaster-related or not.",
    backstory="You're a hybrid AI combining ML and Cohere LLM for disaster tweet classification. "
              "You analyze tweets using a machine learning model and verify the classification using an LLM.",
    allow_delegation=False,
    verbose=True,
    llm=llm
)

response_generator = Agent(
    role="Response Generator",
    goal="Generate appropriate responses based on tweet classification.",
    backstory="You're responsible for crafting clear and informative responses based on disaster classification. "
              "You ensure that disaster alerts are communicated effectively.",
    allow_delegation=False,
    verbose=True,
    llm=llm
)

# Define Tasks
clean_text_task = Task(
    description="Preprocess tweet text by lowercasing and removing punctuation.",
    agent=text_processor,
    expected_output="A cleaned tweet ready for classification.",
    parameters=["tweet"],
    function=lambda tweet: re.sub(f"[{string.punctuation}]", "", tweet.lower())  # Clean text inside the task
)

classify_task = Task(
    description="Classify the cleaned tweet as disaster-related (1) or not (0).",
    agent=disaster_classifier,
    expected_output="A classification label (1 for disaster, 0 for non-disaster).",
    parameters=["cleaned_text"]
)

response_task = Task(
    description="Generate an appropriate response based on the classification result.",
    agent=response_generator,
    expected_output="A text message indicating if a disaster was detected.",
    parameters=["classification"]
)

# Create AI Crew
crew = Crew(
    agents=[text_processor, disaster_classifier, response_generator],
    tasks=[clean_text_task, classify_task, response_task],
    verbose=True
)

# Function to Process Tweets
def classify_tweet(tweet):
    """Processes and classifies a tweet using CrewAI."""

    # Step 1: Run Text Cleaning Task
    cleaned_tweet = text_processor.execute_task(clean_text_task, tweet)

    # Step 2: ML Classification
    vectorized_text = vectorizer.transform([cleaned_tweet])
    ml_prediction = ml_model.predict(vectorized_text)[0]

    # Step 3: LLM Classification
    classification_response = disaster_classifier.execute_task(classify_task, cleaned_tweet)
    try:
        llm_prediction = int(classification_response.strip())
    except (ValueError, AttributeError):
        llm_prediction = 0  # Default to "No Disaster" if LLM fails

    # Step 4: Final Classification Decision
    classification = 1 if ml_prediction == 1 or llm_prediction == 1 else 0

    # Step 5: Generate Response
    return response_generator.execute_task(response_task, classification)

# Example Usage
tweet_example = "Huge earthquake in California! Buildings are collapsing!"
print(classify_tweet(tweet_example))




> Entering new CrewAgentExecutor chain...
Thought: I will ensure the text is lowercase and remove any punctuation to meet the formatting requirements.

Final Answer: huge earthquake in california buildings are collapsing

> Finished chain.


> Entering new CrewAgentExecutor chain...
Thought: This tweet describes a significant earthquake in California, with buildings collapsing, which indicates a potential disaster situation. 

Final Answer: 1

> Finished chain.


> Entering new CrewAgentExecutor chain...
Thought: I will craft a response to indicate a disaster classification.

Final Answer: Disaster detected! This tweet indicates a potential emergency situation and requires immediate attention.

> Finished chain.
Disaster detected! This tweet indicates a potential emergency situation and requires immediate attention.


## Performance Benchmarking

In [ ]:
import time
start_time = time.time()
processed_tweet = text_processor.execute_task(clean_text_task, tweet_example)
end_time = time.time()
print(f"Text Processing Time: {end_time - start_time:.4f} seconds")



> Entering new CrewAgentExecutor chain...
Thought: I will convert the text to lowercase and remove punctuation to ensure it is ready for further analysis.

Final Answer: huge earthquake in california buildings are collapsing

> Finished chain.
Text Processing Time: 1.0878 seconds


In [ ]:
from sklearn.metrics import classification_report

y_true = [1, 0, 1, 1, 0]  # Actual labels
y_pred = [1, 0, 1, 0, 1]  # Predicted labels by AI agents

print(classification_report(y_true, y_pred))

              precision    recall  f1-score   support

           0       0.50      0.50      0.50         2
           1       0.67      0.67      0.67         3

    accuracy                           0.60         5
   macro avg       0.58      0.58      0.58         5
weighted avg       0.60      0.60      0.60         5

